In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score
import xgboost as xgb

# ohe seq input

In [3]:
train_data = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/gene_seq_train.csv')
# train_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data0/res_train.csv')

train_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/res_train.csv')
#don't touch test data, split out validation data from training data during training
test_data = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/gene_seq_test.csv')
# test_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data0/res_test.csv')
test_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/res_test.csv')

## Data prep

In [4]:
# train_data = train_data.drop(['wgs_id'], axis=1) 
# train_data['concatenated'] = train_data.apply(lambda row: ''.join(row), axis=1)

# import pandas as pd
# import numpy as np

# # Example DataFrame with DNA sequences
# train_data = pd.DataFrame({
#     'sequence': ['ATCG']
# })

# # Nucleotide to one-hot encoding mapping
# nt_map = {'A': [1, 0, 0, 0], 'C': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'T': [0, 0, 0, 1], 'N': [0, 0, 0, 0]}

# def one_hot_encode_seq(sequence):
#     """Converts a DNA sequence into a one-hot encoded matrix."""
#     return np.array([nt_map[nt] for nt in sequence])

# # Apply the one-hot encoding to each sequence and store the result in a new column
# train_data['one_hot'] = train_data['concatenated'].apply(one_hot_encode_seq)

# # Optional: Convert the list of matrices in the 'one_hot' column into a 3D numpy array
# # The resulting array shape will be (number of sequences, sequence length, 4)
# one_hot_array = np.stack(train_data['one_hot'].values)

# # print("One-hot encoded array shape:", one_hot_array.shape)
# # print("One-hot encoded data for the first sequence:\n", one_hot_array[0])
# one_hot_array = np.stack(train_data['one_hot'].values)

In [5]:
one_hot_array = np.load('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/seq_ohe_array.npy')


In [6]:
# np.savetxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/seq_ohe_array.csv', one_hot_array, delimiter=',')
# np.save('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/seq_ohe_array.npy', one_hot_array)
one_hot_array = np.load('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/seq_ohe_array.npy')
X_reshaped = one_hot_array.reshape(one_hot_array.shape[0], -1)

nan_rows_count = train_target.isnull().any(axis=1).sum()

print(f"Number of rows containing NaN: {nan_rows_count}")

for column in train_target.columns:
    mode_value = train_target[column].mode()[0]  # Get the mode of each column
    train_target[column].fillna(mode_value, inplace=True)  # Fill NaN with the mode

Number of rows containing NaN: 1330


### Model training

In [7]:
X = X_reshaped
y = train_target.values
y = np.log(y)
print(X.shape)
print(y.shape)

(10849, 227360)
(10849, 13)


In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
# X, y = load_iris(return_X_y=True)

In [7]:
# X = np.random.rand(4, 100).T  # Example data; replace with your actual data
# y = np.random.randint(0, 2, 100)  # Example binary target; replace with your actual target
X = X_reshaped
y = train_target.values
print(X.shape)
print(y.shape)
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Logistic Regression
# print('logistic regression')
# lr_model = LinearRegression()
# lr_model.fit(X_train, y_train)
# lr_predictions = lr_model.predict(X_test)
# lr_accuracy = accuracy_score(y_test, lr_predictions)
# print(f"Logistic Regression Accuracy: {lr_accuracy}")

# XGBoost
print('XGB regression')
xgb_model = xgb.XGBRegressor(use_label_encoder=False, eval_metric='rmse', multi_strategy="multi_output_tree")
# xgb_model = xgb.XGBRegressor(use_label_encoder=False, eval_metric='rmse', tree_method='gpu_hist', multi_strategy="multi_output_tree")
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, xgb_predictions)
print(f"XGBoost RMSE: {mse}")

(10849, 227360)
(10849, 13)


XGB regression
XGBoost RMSE: 13.66698289731201


### Grid search

#### xgb

In [12]:
from sklearn.model_selection import GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define parameter grid for XGBoost
xgb_param_grid = {
    'n_estimators': [50, 100, 200],  # Number of gradient boosted trees. Equivalent to number of boosting rounds
    'max_depth': [3, 4, 5],  # Maximum tree depth for base learners
    'learning_rate': [0.01, 0.1, 0.2],  # Boosting learning rate (xgb’s “eta”)
}

# Create grid search object
# xgb_grid_search = GridSearchCV(xgb.XGBRegressor(use_label_encoder=False, eval_metric='rmse', multi_strategy="multi_output_tree"), xgb_param_grid, cv=5, scoring='accuracy', verbose=1)
xgb_grid_search = GridSearchCV(
    # xgb.XGBRegressor(use_label_encoder=False, eval_metric='rmse', tree_method='hist', device = 'cuda', multi_strategy="multi_output_tree"),
    xgb.XGBRegressor(use_label_encoder=False, eval_metric='rmse', tree_method='hist', multi_strategy="multi_output_tree"),
    xgb_param_grid,
    cv=5,
    scoring='neg_mean_squared_error',  # Changed to a regression scoring metric
    verbose=1
)

xgb_grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best Parameters for XGBoost:", xgb_grid_search.best_params_)
print("Best Score for XGBoost:", xgb_grid_search.best_score_)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


### random forest

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Define parameter grid for Random Forest
rf_param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [3, 4, 5],  # Maximum depth of the tree
    'min_samples_split': [2, 4, 6],  # Minimum number of samples required to split an internal node
}

# Create grid search object for Random Forest
rf_grid_search = GridSearchCV(
    RandomForestRegressor(),
    rf_param_grid,
    cv=5,
    scoring='neg_mean_squared_error',  # Using MSE as the scoring metric
    verbose=1
)

rf_grid_search.fit(X_train, y_train)

# Output best parameters and best score
print("Best Parameters for Random Forest:", rf_grid_search.best_params_)
print("Best Score for Random Forest:", rf_grid_search.best_score_)


# SNP freq input

In [ ]:
train_aa_data = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/aa_data_train.csv',delimiter =',')
# train_aa_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data0/res_train.csv')

train_aa_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/mic_aa_train.csv')
#don't touch test data, split out validation data from training data during training
test_aa_data = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/aa_data_test.csv',delimiter =',')
# test_aa_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data0/res_test.csv')
test_aa_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/mic_aa_test.csv')

## Model training

In [ ]:
# Assuming `X` is your features array of shape (4, n) and `y` is your target array of shape (n,)
# You might need to transpose `X` to get it into the shape (n, 4)
# X = np.random.rand(4, 100).T  # Example data; replace with your actual data
# y = np.random.randint(0, 2, 100)  # Example binary target; replace with your actual target
X = train_aa_data
y = train_aa_target.values
print(X.shape)
print(y.shape)
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Logistic Regression

# aa_XGBoost
print('XGB regression')
aa_xgb_model = xgb.XGBRegressor(use_label_encoder=False, eval_metric='rmse', multi_strategy="multi_output_tree")
aa_xgb_model.fit(X_train, y_train)
aa_xgb_predictions = aa_xgb_model.predict(X_test)
aa_xgb_accuracy = accuracy_score(y_test, aa_xgb_predictions)
print(f"aa_XGBoost Accuracy: {aa_xgb_accuracy}")

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid for aa_XGBoost
aa_xgb_param_grid = {
    'n_estimators': [50, 100, 200],  # Number of gradient boosted trees. Equivalent to number of boosting rounds
    'max_depth': [3, 4, 5],  # Maximum tree depth for base learners
    'learning_rate': [0.01, 0.1, 0.2],  # Boosting learning rate (aa_xgb’s “eta”)
}

# Create grid search object
aa_xgb_grid_search = GridSearchCV(xgb.XGBRegressor(use_label_encoder=False, eval_metric='rmse', multi_strategy="multi_output_tree"), xgb_param_grid, cv=5, scoring='accuracy', verbose=1)
xgb_grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best Parameters for aa_XGBoost:", aa_xgb_grid_search.best_params_)
print("Best Score for aa_XGBoost:", aa_xgb_grid_search.best_score_)


EMB_BINARY_PHENOTYPE
S                       8334
R                       2261
I                       1559
dtype: int64